Centro de Estudos e Sistemas Avançados do Recife

Pós-graduação em Engenharia e Análise de Dados

Disciplina de RNA e Deep Learning

Professor: Silvan Ferreira da Silva Junior 
 
Grupo:
* Anísio Pereira Batista Filho (apbf@cesar.school)
* Carlos Cezar Lopes de Mendonça (cclm@cesar.school)
* Rodolpho Victor França Valsconcelos (rvfv@cesar.school)

2. Construção de um RAG utilizando LangChain


Deverá ser desenvolvido um sistema RAG (Retrieval-Augmented Generation) utilizando a biblioteca LangChain para um ou mais documentos de sua escolha. Podendo ser arquivos PDF, de texto, páginas da web etc. O projeto será avaliado nos seguintes aspectos:

* Escolha do Documento
* Splitting do Documento
* Criação de Vector Store
* Retrieval
* Geração de Respostas

In [1]:
# !pip install pymupdf langchain_community langchain_openai faiss-cpu

In [2]:
from langchain.chains import RetrievalQA
from langchain.llms import OpenAI
from langchain.vectorstores import FAISS
from langchain.embeddings import OpenAIEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain_openai import ChatOpenAI

import fitz  # PyMuPDF

In [3]:
# from google.colab import drive
# drive.mount('/content/drive')

In [4]:
import getpass
import os

os.environ["OPENAI_API_KEY"] = getpass.getpass()

In [5]:
def read_pdf(pdf_path):
    pdf_text = ''
    doc = fitz.open(pdf_path)
    for page_num in range(len(doc)):
        page = doc.load_page(page_num)
        pdf_text += page.get_text()
    return pdf_text

In [6]:

# Leia e divida o documento PDF
pdf_path = 'data/Scrum-Guide.pdf'
text = read_pdf(pdf_path)

# Divida o texto em blocos menores
text_splitter = CharacterTextSplitter(chunk_size=500, chunk_overlap=50)
texts = text_splitter.split_text(text)

# Crie embeddings para os blocos de texto
embeddings = OpenAIEmbeddings()  # Use seu modelo de embeddings aqui
vector_store = FAISS.from_texts(texts, embeddings)

# Configure o modelo de linguagem para gerar respostas
llm = ChatOpenAI(model="gpt-4o-mini")

# Configure o sistema de recuperação e geração de respostas
retriever = vector_store.as_retriever()
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",  # Pode ser "map_reduce" ou outro tipo suportado
    retriever=retriever
)

# Função para responder perguntas
def answer_question(question):
    return qa_chain.run(question)

# Teste do sistema
question = "Qual é o tópico principal do documento?"
answer = answer_question(question)
print(answer)


c:\Users\anisi\anaconda3\envs\cesarschool310\lib\site-packages\langchain_core\_api\deprecation.py:151: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 1.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAIEmbeddings`.
  warn_deprecated(
c:\Users\anisi\anaconda3\envs\cesarschool310\lib\site-packages\langchain_core\_api\deprecation.py:151: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use invoke instead.
  warn_deprecated(


O tópico principal do documento é o Guia do Scrum, que apresenta a definição, teoria, valores, papéis, eventos e artefatos do framework Scrum, visando ajudar pessoas, times e organizações a gerar valor por meio de soluções adaptativas para problemas complexos.
